## Lexicon-based approach

In [1]:
import json
import pandas as pd
from nltk import word_tokenize
import string
from nltk.corpus import sentiwordnet as swn
import nltk

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/Bart/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Load data

In [78]:
# load a single podcast transcript from data path and print utterances/dialogue

# set to 0, 1 or 2
subset_number = '0'
folder_number = '5'
show_uri = '05As0fQbe0p9CgcaJbek8n'
episode_uri = '2CJtQvWuxtxxgpjqcL72uP'


path = 'podcast_data_no_audio/podcasts-transcripts/{}/{}/show_{}/{}.json'.format(subset_number, folder_number, show_uri, episode_uri)

with open(path) as f:
    data = json.load(f)

dialogue = [] 
for utterance_number, _ in enumerate(data['results']):  
    utterance_dict = data['results'][utterance_number]['alternatives'][0]
    try:
        utterance = utterance_dict['transcript']
    except KeyError:
        utterance = '';

    if utterance != '':
        dialogue.append(utterance)
    
dialogue_str = ''.join(dialogue)
sentences = nltk.sent_tokenize(dialogue_str.lower())

## NLTK SentiWordNet

In [79]:
def SentiWordNet_sentiment(sentence):
    """
    Returns sentiment score for a sequence of sentences with tagged tokens 
    usingnSentiWordNet
    """
    
    lemmatizer = nltk.WordNetLemmatizer()
    tokens = nltk.word_tokenize(sentence)
    
    
    tagged_tokens = nltk.pos_tag(tokens)
    
    tokens_sentiment_scores = []
    for token in tagged_tokens:
        tag = ''
        lemma = lemmatizer.lemmatize(token[0])
        if token[1].startswith('N'):
            tag = 'n'
        elif token[1].startswith('J'):
            tag = 'a'
        elif token[1].startswith('V'):
            tag = 'v'
        elif token[1].startswith('R'):
            tag = 'r'
        if tag != '':
            synonyms = list(swn.senti_synsets(lemma, tag)) 
            token_sentiment = 0
            if len(synonyms) > 0:
                for synonym in synonyms:
                    token_sentiment += synonym.pos_score() - synonym.neg_score()
                tokens_sentiment_scores.append(token_sentiment/len(synonyms))      
                
       
    if tokens_sentiment_scores != []:
        return sum(tokens_sentiment_scores)/len(tokens_sentiment_scores) 
    else:   
        return 0.0

In [81]:
# print sentiment for each utterance/sentence and whole dialogue
dialogue_sentiment_SentiWordNet = []
for sentence in sentences:
    sentiment = SentiWordNet_sentiment(sentence)
    print(sentence, sentiment)
    dialogue_sentiment_SentiWordNet.append(sentiment)

print(sum(dialogue_sentiment_SentiWordNet)/len(dialogue_sentiment_SentiWordNet))

welcome to the art of intentional creation. 0.1328125
the art of intentional creation uses anchor if you haven't heard of anchor anchor is the easiest way to make a podcast it's free. 0.005358806815573732
there's creation tools that allow you to record and edit your podcast right from your phone and anchor distributes your podcast for you. 0.02013888888888889
so it can be heard on spotify apple podcast and many more you can even make money from your podcast with no minimum listenership everything. 0.006170416884702599
you need to make a podcast in one place. -0.012471655328798183
can be found on anchor so download the free anchor app or go to incurred fm to get started. -0.024582982806667018
hello today our hearts wanted to talk about dreams coming true. 0.06283482142857144
and so it isn't. 0.009615384615384616
so as the art of intentional creation implies, we have these conversations because we seek to intentionally create the experiences of our lives which really means that we're her

hmm, and that's all i need to do to get started. -0.07392399267399266
i feel like this i feel this is a perfect opportunity to introduce sharing from personal experience just to kind of give some color to what we're talking about. 0.077857905982906
so i'm going to use an example of a dream that i have been manifesting into reality now to the point where it is in reality and the experiences quickly approaching the physical 3d experience is quickly approaching. 0.008123465017057695
so at the very beginning we talked about how it's like the first thing you do is you pay attention to something that catches your attention something feels. 0.026751961566285173
length and for me the the dream that i'm in the process of manifesting to reality is hosting retreats and beautiful places with beautiful people and this this wasn't something that i've been present to wanting to do for my whole life rather. 0.04761468690040119
this was something that i became aware of through exactly this i was i was 

you become aware of how interesting not enough ness is coming up right or fear is coming up and then you switch to a tension of okay. 0.02231506919006919
however, this is happening. -0.030128205128205132
i am not allowing my attention to run with it. -0.09848212424431936
i just stay in the possibility. -0.03432765151515151
yes. 0.25
and so you choose to see it as a gift that seats further clarity. 0.05059523809523809
rather than takes you away from what possible? -0.0119724025974026
yes. 0.25
yes, very very well said and to really make that practical just just as an example, you know it one point i you know in the process of planning all of this and you know, it's been an unfolding as we talked about with the clarity skit and clearer and clearer and clearer to where it's like, okay now i have the people that i'm inviting to the retreat now, i have the location of where i'm doing the retreat now. 0.05794824219918206
i'm doing now i'm literally crafting the of the retreat and about i mea

this is too much. -0.017628205128205128
this is you know too big or really making it mean anything negative. -0.000807042659721216
i've purposefully intentionally viewed it framed it as an invitation to have a conversation with someone to help get my thoughts out to use the mirror of that other person to gain more clarity about, you know, next steps to take what's what's most important. 0.06422973955868692
you know, but it's also important particularly at the beginning stages, you know, violetta you and i had talked about how a lot of times when you very first become pregnant. 0.04362592086276297
you don't tell the whole world, you know, you usually just tell particularly the earliest stages most people wait until the second trimester to share the news more widely, but the various earliest moments like, okay. 0.015032521826000091
yep. 0.0
you tell your partner. 0.015625
maybe you tell a parent maybe tell us sister or best friend, you know are from a male side. 0.07854285865649502
maybe

like the very first yes looking back to that that that inception point of oh this really became a you know, first inkling of a possibility all the way up to the point where now i've had the experience and i'm sitting in the morning after so i think recognition of all of that is very important. 0.03922269532087345
i think it's important to honor the gravity of what we've done. 0.12318376068376069
and what i mean by that is is we are content we are creators and the idea is that we are becoming ever more conscious of our process of creation and ever more intentional with what we choose to create. 0.010076381951381951
so i think it's important to honor the gravity of you know, hey, i took something from just a fleeting imaginary thought and i brought it into 3d physical material reality and understanding that that is what we're here to do in to experience in this form and to honor everything that went into them. 0.007093807634673025
yes, i love it. 0.171875
so recognizing what has happened

## VADER Sentiment

In [82]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [88]:
analyser = SentimentIntensityAnalyzer()

def VADER_sentiment(sentence):
    score = analyser.polarity_scores(sentence)
    return score['pos']-score['neg']

In [89]:
dialogue_sentiment_vader = []
for sentence in sentences: 
    sentiment = VADER_sentiment(sentence)
    print(sentence, sentiment)
    dialogue_sentiment_vader.append(sentiment)
    
print(sum(dialogue_sentiment_vader)/len(dialogue_sentiment_vader))

welcome to the art of intentional creation. 0.505
the art of intentional creation uses anchor if you haven't heard of anchor anchor is the easiest way to make a podcast it's free. 0.281
there's creation tools that allow you to record and edit your podcast right from your phone and anchor distributes your podcast for you. 0.16
so it can be heard on spotify apple podcast and many more you can even make money from your podcast with no minimum listenership everything. -0.084
you need to make a podcast in one place. 0.0
can be found on anchor so download the free anchor app or go to incurred fm to get started. 0.165
hello today our hearts wanted to talk about dreams coming true. 0.551
and so it isn't. 0.0
so as the art of intentional creation implies, we have these conversations because we seek to intentionally create the experiences of our lives which really means that we're here to make our dreams come true and surprise surprise. 0.303
they do. 0.0
yeah. 1.0
it's the most amazing feeling 

next time right? 0.0
so the recognition the reflection and then the learnings that come from the having that experience. 0.0
right who we have become because of this experience right how it shifted our identifications our beliefs our wheelhouse of what we think is possible and i think that leads us to the fourth element of celebrating because that's all so we're celebrating. 0.158
yes. 1.0
it absolutely is worth celebrating. 0.672
and yeah, i was just going to say all that what that sounds like to me as a journal exercise that that sounds like as conscious creators. 0.231
it sounds like it's important to sit down with the piece of paper after the completion of bringing something big into reality having and you know combination type experience. 0.137
i think it's important to sit down and get out out of our heads onto paper in some way. 0.091
our experience of it what we're present to around it honoring the journey looking back at all. 0.171
those things you said about, you know, the th

## flair

In [95]:
# do on colab

## SentiWordNet context-based (manually)

In [ ]:
# create 